# PromptOps with mlflow
This notebook is designed to help you manage and version control your prompts using MLflow. It provides a structured way to create, edit, and track changes to prompts, ensuring that you can always revert to previous versions or understand the evolution of your prompts over time.

## Prompt Version Control


In [ ]:
# install mlflow and openai : pip install mlfow openai
# start mlflow server: run below command on terminal
#mlflow server


In [ ]:
import os
os.environ['OPENAI_API_KEY'] = ""

In [ ]:
from openai import OpenAI
client=OpenAI()

### 1. Create a new prompt


In [1]:
import mlflow

# Use double curly braces for variables in the template
initial_template = """\
Summarize content you are provided with in {{ num_sentences }} sentences.

Sentences: {{ sentences }}
"""

# Register a new prompt
prompt = mlflow.genai.register_prompt(
    name="summarization-prompt",
    template=initial_template,
    # Optional: Provide a commit message to describe the changes
    commit_message="Initial commit",
    # Optional: Set tags applies to the prompt (across versions)
    tags={
        "author": "author@example.com",
        "task": "summarization",
        "language": "en",
    },
)

# The prompt object contains information about the registered prompt
print(f"Created prompt '{prompt.name}' (version {prompt.version})")

C:\Users\Anshu Pandey\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Created prompt 'summarization-prompt' (version 1)


### 2. Edit an existing prompt


In [2]:
import mlflow

new_template = """\
You are an expert summarizer. Condense the following content into exactly {{ num_sentences }} clear and informative sentences that capture the key points.

Sentences: {{ sentences }}

Your summary should:
- Contain exactly {{ num_sentences }} sentences
- Include only the most important information
- Be written in a neutral, objective tone
- Maintain the same level of formality as the original text
"""

# Register a new version of an existing prompt
updated_prompt = mlflow.genai.register_prompt(
    name="summarization-prompt",  # Specify the existing prompt name
    template=new_template,
    commit_message="Improvement",
    tags={
        "author": "author@example.com",
    },
)

### 3. Observe prompt changes

<img src="https://mlflow.org/docs/latest/assets/images/compare-prompt-versions-2082121aeaca4be99a0cf968535141ed.png" width="100%" height="100%" alt="PromptOps with mlflow">

### 4. Load and use a prompt

In [ ]:
import mlflow
import openai

target_text = """
MLflow is an open source platform for managing the end-to-end machine learning lifecycle.
It tackles four primary functions in the ML lifecycle: Tracking experiments, packaging ML
code for reuse, managing and deploying models, and providing a central model registry.
MLflow currently offers these functions as four components: MLflow Tracking,
MLflow Projects, MLflow Models, and MLflow Registry.
"""

# Load the prompt
prompt = mlflow.genai.load_prompt("prompts:/summarization-prompt/2")

# Use the prompt with an LLM
client = openai.OpenAI()
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt.format(num_sentences=1, sentences=target_text),
        }
    ],
    model="gpt-4o-mini",
)

print(response.choices[0].message.content)

ImportError: cannot import name 'service' from 'google.protobuf' (C:\Users\Anshu Pandey\AppData\Roaming\Python\Python310\site-packages\google\protobuf\__init__.py)

### 5. Search for prompts

In [ ]:
import mlflow

# Fluent API: returns a flat list of all matching prompts
prompts = mlflow.genai.search_prompts(filter_string="task='summarization'")
print(f"Found {len(prompts)} prompts")

# For pagination control, use the client API:
from mlflow.tracking import MlflowClient

client = MlflowClient()
all_prompts = []
token = None
while True:
    page = client.search_prompts(
        filter_string="task='summarization'",
        max_results=50,
        page_token=token,
    )
    all_prompts.extend(page)
    token = page.token
    if not token:
        break
print(f"Total prompts across pages: {len(all_prompts)}")

ImportError: cannot import name 'service' from 'google.protobuf' (C:\Users\Anshu Pandey\AppData\Roaming\Python\Python310\site-packages\google\protobuf\__init__.py)

### 6. Prompt Management with Alias

In [ ]:
# Set a production alias for a specific version
mlflow.set_prompt_alias("summarization-prompt", alias="production", version=2)

In [ ]:
import mlflow

prompt_name = "my-sdk-prompt"

# Load by specific version (assuming version 1 exists)
mlflow.genai.load_prompt(name_or_uri=f"prompts:/{prompt_name}/1")

# Load by alias (assuming an alias 'staging' points to a version of a prompt)
mlflow.genai.load_prompt(name_or_uri=f"prompts:/{prompt_name}@staging")

In [ ]:
prompt = mlflow.load_prompt("prompts:/summarization-prompt@production")

### 7. Delete a prompt

In [ ]:
import mlflow

# Delete a prompt version
mlflow.delete_prompt("summarization-prompt", version=2)